The model combines the 10 team-based and the 38 game-based features, for a total of 48 features. We will use this combined information for predicting the outcomes of more than 9000 games. More precisely, we will see whether the home team wins, the away team wins, or the game ends in a tie.

In [0]:
import numpy as pd
import pandas as pd
from collections import Counter
from copy import copy
import tensorflow as tf
from keras import Sequential
from keras.layers import Input, Dense, Dropout
from keras.models import Model

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
# Read game_based dataset
dataset_fp = "drive/Team Drives/Deep Learning Project/ann/aggregated_game_based_36_features.csv"
df1 = pd.read_csv(dataset_fp)
df1.head(5)

,Corner_Kick0,Corner_Kick1,Failed_Through_Ball0,Failed_Through_Ball1,First_Yellow_Card0,First_Yellow_Card1,Foul0,Foul1,Free_Kick_Awarded0,Free_Kick_Awarded1,...,Straight_Red_Card1,Substitution0,Substitution1,assist_method0,assist_method1,fast_break0,fast_break1,id_odsp,is_goal0,is_goal1
0,4,2,0,0,0,1,5,6,5,5,...,0,0,1,10,1,1,0,UFot0hit/,2,0
1,2,2,0,0,1,2,15,8,9,15,...,0,0,1,3,5,0,0,Aw5DflLH/,0,0
2,2,2,0,0,0,1,5,15,15,7,...,0,1,1,7,1,0,0,bkjpaC6n/,0,0
3,2,2,0,0,1,1,4,10,11,5,...,0,0,0,6,12,0,0,CzPV312a/,1,0
4,1,5,0,0,0,1,6,6,6,6,...,0,0,0,7,9,0,0,GUOdmtII/,1,0


In [0]:
# add the league feature
dataset_fp = "drive/Team Drives/Deep Learning Project/ginf.csv"
df3 = pd.read_csv(dataset_fp)
df3.head()

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN


In [0]:
# Read game_based dataset
dataset_fp = "drive/Team Drives/Deep Learning Project/ann/team based data.csv"
df2 = pd.read_csv(dataset_fp)
df2.head(5)

,#games,goals,losing scores,loss,team,tie,wins,winning rate,loss rate,average goals,average losing score,tie rate
0,179,192,300,89,Hamburg SV,42,48,0.268156,0.497207,1.072626,1.675978,0.234637
1,180,392,197,37,Borussia Dortmund,37,106,0.588889,0.205556,2.177778,1.094444,0.205556
2,178,204,246,73,FC Augsburg,49,56,0.314607,0.410112,1.146067,1.382022,0.275281
3,132,162,205,55,SC Freiburg,35,42,0.318182,0.416667,1.227273,1.553030,0.265152
4,179,254,348,81,Werder Bremen,49,49,0.273743,0.452514,1.418994,1.944134,0.273743


Then we aggregate the two datasets

In [0]:
# create the game-based result dataset
import pandas as pd

df = pd.read_csv("drive/Team Drives/Deep Learning Project/events.csv")
df = df.ix[:,[0,8,9,16]]
game = df['id_odsp'].unique()
game_base = df.groupby('id_odsp')

game_dic = {}
for i in game:
    game_dic[i]=0
    
for each in game:
    this_game = game_base.get_group(each)
    this_game_goal = this_game.groupby('is_goal')


    if len(this_game['is_goal'].unique()) ==2:
        goal = this_game_goal.get_group(1)
        if len(goal['event_team'].unique())==2:
            a = dict(Counter(goal['event_team']))
            temp = copy(a)
            if list(temp.values())[0] > list(temp.values())[1]:
                a['win']=list(temp.keys())[0]
                a['loss']=list(temp.keys())[1]
                a['draw']=None
            else:
                if list(temp.values())[0] == list(temp.values())[1]:
                        a['win']=None
                        a['loss']=None
                        a['draw']=list(temp.keys())
                else:
                    a['win']=list(temp.keys())[1]
                    a['loss']=list(temp.keys())[0]
                    a['draw']=None
            game_dic[each]=a
        else:
            b = dict(Counter(goal['event_team']))
            temp = copy(b)
            other_team = list(this_game['event_team'].unique())
            for i in other_team:
                if list(temp.keys())[0] != i:
                    b[i]=0
                    opponent = i
            b['win'] = list(temp.keys())[0]
            b['loss'] = opponent
            b['draw'] = None
            game_dic[each]=b
    else:
        team = this_game['event_team'].unique()
        c = {}
        for i in team:

            c[i]=0
        temp = copy(c)
        c['win']=None
        c['loss']=None
        c['draw']=list(temp.keys())
        game_dic[each]=c



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [0]:
# get the result dataset and then aggregate all the data set. We have 25 features in total. For each game, add the wining rate, loss rate, tie rate, avg lossing score, avg goal for the team and opponent
id_odsp = list(game_dic.keys())
result = []
team = []
opponent = []
winning_rate0 = []
loss_rate0= []
average_goal0 = []
average_lossing_score0 =[]
tie_rate0 = []
winning_rate1 = []
loss_rate1= []
average_goal1 = []
tie_rate1 = []
average_lossing_score1 = []
for id in id_odsp:
    game = game_dic[id]
    team_temp = list(game.keys())[0]
    opponent_temp = list(game.keys())[1]
    team.append(team_temp)
    opponent.append(opponent_temp)
    if team_temp == game['win']:
        result.append(0)
    else:
        if opponent_temp == game['win']:
            result.append(1)
        else:
            result.append(2)
for i in team:
    temp = df2[df2['team']==i]
    winning_rate0.append(float(temp['winning rate']))
    loss_rate0.append(float(temp['loss rate']))
    average_goal0.append(float(temp['average goals']))
    average_lossing_score0.append(float(temp['average losing score']))
    tie_rate0.append(float(temp['tie rate']))
for j in opponent:
    temp = df2[df2['team']==j]
    winning_rate1.append(float(temp['winning rate']))
    loss_rate1.append(float(temp['loss rate']))
    average_goal1.append(float(temp['average goals']))
    average_lossing_score1.append(float(temp['average losing score']))
    tie_rate1.append(float(temp['tie rate']))
    
target = pd.DataFrame({'id_odsp':id_odsp, 'result':result, 'team':team, 'opponent':opponent, 'wining_rate0':winning_rate0, 'loss_rate0':loss_rate0
                      ,'average_goal0':average_goal0,'average_lossing_score0':average_lossing_score0, 'tie_rate0':tie_rate0,'wining_rate1':winning_rate1, 'loss_rate1':loss_rate1
                      ,'average_goal1':average_goal1,'average_lossing_score1':average_lossing_score1, 'tie_rate1':tie_rate1})
target.head(5)


,average_goal0,average_goal1,average_lossing_score0,average_lossing_score1,id_odsp,loss_rate0,loss_rate1,opponent,result,team,tie_rate0,tie_rate1,wining_rate0,wining_rate1
0,2.177778,1.072626,1.094444,1.675978,UFot0hit/,0.205556,0.497207,Hamburg SV,0,Borussia Dortmund,0.205556,0.234637,0.588889,0.268156
1,1.227273,1.146067,1.553030,1.382022,Aw5DflLH/,0.416667,0.410112,FC Augsburg,2,SC Freiburg,0.265152,0.275281,0.318182,0.314607
2,1.418994,0.666667,1.944134,1.575758,bkjpaC6n/,0.452514,0.575758,Kaiserslautern,0,Werder Bremen,0.273743,0.303030,0.273743,0.121212
3,1.195238,2.149038,1.480952,0.750000,CzPV312a/,0.433333,0.105769,Paris Saint-Germain,0,Lorient,0.266667,0.211538,0.300000,0.682692
4,1.160305,1.107143,1.519084,1.500000,GUOdmtII/,0.458015,0.500000,Valenciennes,0,Caen,0.229008,0.241071,0.312977,0.258929


In [0]:
# merge the data to get the final dataset
final2 = pd.merge(df1,target)
final2.head(10)

,Corner_Kick0,Corner_Kick1,Failed_Through_Ball0,Failed_Through_Ball1,First_Yellow_Card0,First_Yellow_Card1,Foul0,Foul1,Free_Kick_Awarded0,Free_Kick_Awarded1,...,average_lossing_score1,loss_rate0,loss_rate1,opponent,result,team,tie_rate0,tie_rate1,wining_rate0,wining_rate1
0,4,2,0,0,0,1,5,6,5,5,...,1.675978,0.205556,0.497207,Hamburg SV,0,Borussia Dortmund,0.205556,0.234637,0.588889,0.268156
1,2,2,0,0,1,2,15,8,9,15,...,1.382022,0.416667,0.410112,FC Augsburg,2,SC Freiburg,0.265152,0.275281,0.318182,0.314607
2,2,2,0,0,0,1,5,15,15,7,...,1.575758,0.452514,0.575758,Kaiserslautern,0,Werder Bremen,0.273743,0.303030,0.273743,0.121212
3,2,2,0,0,1,1,4,10,11,5,...,0.750000,0.433333,0.105769,Paris Saint-Germain,0,Lorient,0.266667,0.211538,0.300000,0.682692
4,1,5,0,0,0,1,6,6,6,6,...,1.500000,0.458015,0.500000,Valenciennes,0,Caen,0.229008,0.241071,0.312977,0.258929
5,3,0,0,0,1,1,15,12,13,15,...,1.479452,0.447917,0.438356,Hertha Berlin,0,Nurnberg,0.260417,0.232877,0.291667,0.328767
6,7,0,0,0,2,1,9,6,6,12,...,1.310811,0.470199,0.500000,Brest,2,Evian Thonon Gaillard,0.238411,0.283784,0.291391,0.216216
7,3,6,0,0,0,1,9,7,8,9,...,1.594595,0.397129,0.450450,AC Ajaccio,0,Toulouse,0.282297,0.351351,0.320574,0.198198
8,8,5,0,0,0,0,6,7,7,6,...,1.082524,0.354369,0.266990,Lyon,1,Nice,0.247573,0.208738,0.398058,0.524272
9,2,6,0,0,2,1,7,6,7,8,...,1.400000,0.329609,0.373913,FC Cologne,0,VfL Wolfsburg,0.256983,0.339130,0.413408,0.286957


In [0]:
# set up the training and test set
del final2['id_odsp']
del final2['team']
del final2['opponent']
from sklearn.model_selection import train_test_split
X= final2.drop('result', axis =1)
Y = final2['result']

X_train, X_test, y_train, y_test = train_test_split(X.values, Y.values, test_size=0.33, random_state=42)

models

In [0]:
len(X.columns)

46

In [0]:
X_train

array([[1.        , 1.        , 0.        , ..., 0.31578947, 0.74747475,
        0.21052632],
       [4.        , 0.        , 0.        , ..., 0.28571429, 0.3358209 ,
        0.31578947],
       [5.        , 3.        , 0.        , ..., 0.27272727, 0.73076923,
        0.31100478],
       ...,
       [0.        , 7.        , 0.        , ..., 0.24757282, 0.3       ,
        0.39805825],
       [4.        , 3.        , 0.        , ..., 0.29310345, 0.1981982 ,
        0.20689655],
       [2.        , 0.        , 0.        , ..., 0.28229665, 0.68269231,
        0.32057416]])

In [0]:
from keras.callbacks import TensorBoard
from keras.layers import Dense, SimpleRNN, LSTM, Embedding
from keras.layers import Input, Conv2D, MaxPooling2D, MaxPool2D, GlobalMaxPooling1D
from keras.layers import UpSampling2D, Dropout, Activation, Flatten
from keras.models import Model, Sequential
from keras import backend as K
import tensorflow as tf
import keras
import numpy as np
from keras.utils import np_utils
%matplotlib inline
from matplotlib import pyplot as plt
from keras import regularizers
from string import printable
from keras.callbacks import TensorBoard
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [0]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
Y = to_categorical(Y)

In [0]:

import keras

# Set parameters
input_number = 46
# --- Setting up a Sigmoid Sequential Model ---
# Initialize model
model = Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(Dense(128, input_shape = (input_number,), activation='relu'))
model.add(Dropout(0.2))
# Add another:
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
#model.add(Dense(128, activation='relu'))
##model.add(Dropout(0.2))
##model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
# Add a softmax layer with 10 output units:
model.add(Dense(3, activation='softmax'))
# Check model summary
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0005), metrics=['accuracy'])
history = model.fit(X,Y, epochs =20,batch_size = 32, validation_split = 0.2)             


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               6016      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 22,915
Trainable params: 22,915
Non-trainable params: 0
_________________________________________________________________
Train on 7259 samples, validate on 1815 samples
Epoch 1/20
7259/7259 [==============================] - 3s 482us/step - loss: 0.9282 - acc

In [0]:
scores = model.evaluate(X_test, y_test) 
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

2995/2995 [==============================] - 0s 52us/step
acc: 69.05%


In [0]:
from tqdm import tqdm
# PREDICTIONS FOR TOP TEAMS
top_ten = ['Barcelona',
 'Juventus',
 'Real Madrid',
 'Paris Saint-Germain',
 'Bayern Munich',
 'Atletico Madrid',
 'Napoli',
 'AS Roma',
 'Lyon',
 'Borussia Dortmund']
final3 = pd.merge(df1,target)
del final3['id_odsp']

final3_10 = final3
final3_5 = final3
final3_3 = final3
final3_1 = final3

for i in tqdm(range(len(final3))):
    if final3['team'][i] not in top_ten and final3['opponent'][i] not in top_ten:
        final3_10 = final3_10.drop(i)

for i in tqdm(range(len(final3))):
    if final3['team'][i] not in top_ten[0:5] and final3['opponent'][i] not in top_ten[0:5]:
        final3_5 = final3_5.drop(i)
for i in tqdm(range(len(final3))):
    if final3['team'][i] not in top_ten[0:3] and final3['opponent'][i] not in top_ten[0:3]:
        final3_3 = final3_3.drop(i)
for i in tqdm(range(len(final3))):
    if final3['team'][i] not in top_ten[0] and final3['opponent'][i] not in top_ten[0]:
        final3_1 = final3_1.drop(i)
        

100%|██████████| 9074/9074 [00:12<00:00, 709.30it/s] 


In [0]:

del final3_10['team']
del final3_10['opponent']
del final3_5['team']
del final3_5['opponent']
del final3_3['team']
del final3_3['opponent']
del final3_1['team']
del final3_1['opponent']
from sklearn.model_selection import train_test_split
final3_10_ = final3_10.reset_index(drop = True)
final3_5_ = final3_5.reset_index(drop = True)
final3_3_ = final3_3.reset_index(drop = True)
final3_1_ = final3_1.reset_index(drop = True)

m = [final3_10_, final3_5_, final3_3_, final3_1_]
name = ['10','5','3','1']
for j in range(4):

  X= m[j].drop('result', axis =1)
  Y = m[j]['result']

  Y = to_categorical(Y)
  score = model.evaluate(X,Y)
  print('For top', name[j],'teams' )
  print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))




1910/1910 [==============================] - 0s 51us/step
For top 10 teams
acc: 76.96%
985/985 [==============================] - 0s 47us/step
For top 5 teams
acc: 80.00%
598/598 [==============================] - 0s 56us/step
For top 3 teams
acc: 79.77%
201/201 [==============================] - 0s 85us/step
For top 1 teams
acc: 79.10%
